# MLFlow-ൽ Phi-3 മോഡൽ ഓടിക്കല്‍


---------
ഈ നോട്ട്‌ബുക്ക് Phi-3 mini 4K Instruct എന്ന ഉദാഹരണത്തിൽ Transformer pipeline ഉപയോഗിക്കുന്നതിനെ വിശദീകരിക്കുന്നു.


In [1]:
# Importing required packages
import mlflow
import transformers

### HuggingFace-ൽ നിന്ന് Phi-3 Mini 4K തിരഞ്ഞെടുക്കൽ


In [2]:
# Selecting Phi-3 HuggingFace model
pipeline = transformers.pipeline(
    task = "text-generation",
    model = "microsoft/Phi-3-mini-4k-instruct"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### MLFlow ആർട്ടിഫാക്ട് സൃഷ്ടിക്കുന്നു


In [3]:
# Setting up MLflow model's configuration
model_config = {
    "max_length": 300,
    "truncation": True,
    "include_prompt": False,
}

In [ ]:
# Generating MLflow artifact from HuggingFace model
with mlflow.start_run():
    model_info = mlflow.transformers.log_model(
        transformers_model = pipeline,
        artifact_path = "phi3-mlflow-model",
        model_config = model_config,
        task = "llm/v1/chat"
    )

### Phi-3 നെ MLFlow മോഡലായി ഓടിക്കുന്നു


In [5]:
# Loading Phi-3 MLFlow model
loaded_model = mlflow.pyfunc.load_model(
    model_uri = model_info.model_uri
)

Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
loaded_model.metadata.signature

inputs: 
  ['messages': Array({content: string (required), name: string (optional), role: string (required)}) (required), 'temperature': double (optional), 'max_tokens': long (optional), 'stop': Array(string) (optional), 'n': long (optional), 'stream': boolean (optional)]
outputs: 
  ['id': string (required), 'object': string (required), 'created': long (required), 'model': string (required), 'choices': Array({finish_reason: string (required), index: long (required), message: {content: string (required), name: string (optional), role: string (required)} (required)}) (required), 'usage': {completion_tokens: long (required), prompt_tokens: long (required), total_tokens: long (required)} (required)]
params: 
  None

In [7]:
# Testing Phi-3 MLFlow model's inference
messages = [{"role": "user", "content": "What is the capital of Spain?"}]
response = loaded_model.predict(
    {"messages": messages}
)
print(response)

You are not running the flash-attention implementation, expect numerical differences.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'id': 'f9c2ca5c-b684-4a77-8a43-356b671fd797', 'object': 'chat.completion', 'created': 1718709547, 'model': 'microsoft/Phi-3-mini-4k-instruct', 'usage': {'prompt_tokens': 11, 'completion_tokens': 73, 'total_tokens': 84}, 'choices': [{'index': 0, 'finish_reason': 'stop', 'message': {'role': 'assistant', 'content': 'The capital of Spain is Madrid. It is the largest city in Spain and serves as the political, economic, and cultural center of the country. Madrid is located in the center of the Iberian Peninsula and is known for its rich history, art, and architecture, including the Royal Palace, the Prado Museum, and the Plaza Mayor.'}}]}]


In [8]:
# "Beautified" output
print(f"Question: {messages[0]['content']}\n")
print(f"Response: {response[0]['choices'][0]['message']['content']}\n")
print(f"Usage: {response[0]['usage']}")

Question: What is the capital of Spain?

Response: The capital of Spain is Madrid. It is the largest city in Spain and serves as the political, economic, and cultural center of the country. Madrid is located in the center of the Iberian Peninsula and is known for its rich history, art, and architecture, including the Royal Palace, the Prado Museum, and the Plaza Mayor.

Usage: {'prompt_tokens': 11, 'completion_tokens': 73, 'total_tokens': 84}


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
ഡിസ്‌ക്ലെയിമർ:
ഈ രേഖ AI വിവർത്തന സേവനമായ [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. ഞങ്ങൾ കൃത്യതയ്ക്ക് ശ്രമിച്ചെങ്കിലും യാന്ത്രിക വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ അചരത്വങ്ങൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിനാൽ, അതിന്റെ സ്വഭാഷയിൽ ഉള്ള മൂല രേഖയെയാണ് അധികാരപരമായ ഉറവിടമായി പരിഗണിക്കുന്നത്. സുപ്രധാനമായ വിവരങ്ങൾക്ക് പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനത്തിന്റെ ഉപയോഗം മൂലമുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണങ്ങൾക്കോ തെറ്റായ വ്യാഖ്യാനങ്ങളിലേക്കുള്ള ബാധ്യതയ്ക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
